In [8]:
#here we are implementing chi-square testing for feature selection
#how it is done in selectkbest

In [21]:
import pandas as pd
import numpy as np
from google.colab import drive
from scipy import stats
from sklearn.feature_selection import SelectKBest, chi2

In [22]:
df=pd.read_csv('heart_RandomForest.csv')

In [23]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [24]:
df=df[['sex','cp','fbs','target']]
df.head()

,sex,cp,fbs,target
0,1,0,0,0
1,1,0,1,0
2,1,0,0,0
3,1,0,0,0
4,0,0,1,0


In [25]:
sex_disease_crosstab = pd.crosstab(df['sex'], df['target'],
                                      margins=True)
sex_disease_crosstab

target,0,1,All
sex,,,
0,86,226,312
1,413,300,713
All,499,526,1025


In [26]:
cp_disease_crosstab = pd.crosstab(df['cp'], df['target'],
                                      margins=True)
cp_disease_crosstab

target,0,1,All
cp,,,
0,375,122,497
1,33,134,167
2,65,219,284
3,26,51,77
All,499,526,1025


In [27]:
fbs_disease_crosstab = pd.crosstab(df['fbs'], df['target'],
                                      margins=True)
fbs_disease_crosstab

target,0,1,All
fbs,,,
0,417,455,872
1,82,71,153
All,499,526,1025


In [28]:
#dof is degree of freedom
def check_categorical_dependency(crosstab_table, confidence_interval):
    stat, p, dof, expected = stats.chi2_contingency(crosstab_table)
    print ("Chi-Square Statistic value = {}".format(stat))
    print ("P - Value = {}".format(p))
    alpha = 1.0 - confidence_interval
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
	      print('Independent (fail to reject H0)')
    return expected

In [29]:
exp_table_1 = check_categorical_dependency(sex_disease_crosstab, 0.95)

Chi-Square Statistic value = 80.073690160122
P - Value = 1.6803241672263068e-16
Dependent (reject H0)


In [30]:
pd.DataFrame(exp_table_1)

,0,1,2
0,151.890732,160.109268,312.0
1,347.109268,365.890732,713.0
2,499.000000,526.000000,1025.0


In [31]:
exp_table_2 = check_categorical_dependency(cp_disease_crosstab, 0.95)

Chi-Square Statistic value = 280.98224857035257
P - Value = 4.566481371451617e-56
Dependent (reject H0)


In [32]:
pd.DataFrame(exp_table_2)

,0,1,2
0,241.954146,255.045854,497.0
1,81.300488,85.699512,167.0
2,138.259512,145.740488,284.0
3,37.485854,39.514146,77.0
4,499.000000,526.000000,1025.0


In [34]:
exp_table_3 = check_categorical_dependency(fbs_disease_crosstab, 0.95)
pd.DataFrame(exp_table_3)

#here p value is grreater than 0.05 so H0 hypothesis not rejected
#that means no pattern found in  fbs and target

Chi-Square Statistic value = 1.7367985808714619
P - Value = 0.7840227661779474
Independent (fail to reject H0)


,0,1,2
0,424.515122,447.484878,872.0
1,74.484878,78.515122,153.0
2,499.000000,526.000000,1025.0


In [35]:
#Feature Selection using Chi-Square with selectKBest

In [36]:
X = df[["sex",	"cp", "fbs"]]
y = df[["target"]]

In [37]:
selectkbest=SelectKBest(chi2, k=1)
selectkbest.fit(X, y)

SelectKBest(k=1, score_func=<function chi2 at 0x7cec839d9ea0>)

In [38]:
X_new=selectkbest.transform(X)

In [39]:
X_new.shape

(1025, 1)

In [40]:
pd.crosstab(np.squeeze(X_new), np.squeeze(y))
#np.squeeze convert any dimensional to 1d np array

target,0,1
row_0,,
0,375,122
1,33,134
2,65,219
3,26,51


In [41]:
#it is of the cp that has highest chi square statistic value

In [ ]:
#selectkbest is doing like taking 1 feature and calc. chi square test
#with target, calc chi square value and p value and do with all feature
#then keep thode in ascending order of p value and select the first k
#values